In [1]:
import pandas as pd
import requests
import time

# Dicionário para guardar os resultados da API e evitar consultas repetidas
cache = {}

# Conjunto para armazenar os CEPs que não foram encontrados após as tentativas
not_found_ceps = set()

def get_address(cep):
    """
    Consulta a API ViaCEP para obter bairro e cidade a partir do CEP informado.
    Retorna uma tupla (bairro, cidade) ou (None, None) se não conseguir encontrar o endereço.
    Tenta reconsultar a API até 3 vezes caso não obtenha sucesso.
    """
    # Converte o CEP para string, remove hífens e espaços
    cep = str(cep).replace("-", "").strip()
    # Garante que o CEP possui 8 dígitos, preenchendo com zeros à esquerda se necessário
    cep = cep.zfill(8)
    
    # Se já consultou esse CEP, retorna do cache
    if cep in cache:
        return cache[cep]
    
    max_attempts = 3
    result = (None, None)
    for attempt in range(max_attempts):
        try:
            url = f"https://viacep.com.br/ws/{cep}/json/"
            response = requests.get(url)
            if response.status_code == 200:
                data = response.json()
                if "erro" not in data:
                    result = (data.get("bairro"), data.get("localidade"))
                    break  # Sai do loop se a consulta for bem-sucedida
                else:
                    result = (None, None)
            else:
                result = (None, None)
        except Exception as e:
            print(f"Erro ao consultar o CEP {cep}, tentativa {attempt+1}: {e}")
            result = (None, None)
        # Aguarda 1 segundo antes de tentar novamente (opcional)
        time.sleep(1)
    
    # Armazena o resultado no cache
    cache[cep] = result
    
    # Se após as tentativas o CEP não foi encontrado, adiciona ao conjunto de não encontrados
    if result == (None, None):
        not_found_ceps.add(cep)
        
    return result

# Lê o arquivo Excel, garantindo que a coluna do CEP seja interpretada como string
df = pd.read_excel('Clientes Gerais.xlsx', dtype={'Código postal (CEP) de envio': str})

# Garante que os CEPs estejam sem espaços extras
df["Código postal (CEP) de envio"] = df["Código postal (CEP) de envio"].apply(lambda x: str(x).strip())

# Aplica a função get_address para criar as colunas 'Bairro' e 'Cidade'
df["Bairro"], df["Cidade"] = zip(*df["Código postal (CEP) de envio"].apply(get_address))

# Exporta o DataFrame final para um arquivo Excel (.xlsx)
df.to_excel("Relatorio_de_vendas_com_endereco.xlsx", index=False)

# Printa um aviso se houver CEPs que não foram encontrados após as tentativas
if not_found_ceps:
    print("CEP(s) não encontrados após tentativas:", ", ".join(sorted(not_found_ceps)))
else:
    print("Todos os CEPs foram encontrados.")

print("Processamento finalizado. Planilha atualizada salva como 'Relatorio_de_vendas_com_endereco.xlsx'.")


CEP(s) não encontrados após tentativas: 00001000, 00007000, 00022790, 00024000, 00024800, 00026100, 00030000, 00036570, 05.101210, 06730000, 12511220, 15900 021, 17690000, 20061020, 37190000, 40325085, 40325100, 83750000, 86340000, 876000000, 88704200, 91310002, Desconhecido
Processamento finalizado. Planilha atualizada salva como 'Relatorio_de_vendas_com_endereco.xlsx'.


In [2]:
import pandas as pd
import requests
import time

# Dicionário para guardar os resultados da API e evitar consultas repetidas
cache = {}

# Conjunto para armazenar os CEPs que não foram encontrados após as tentativas
not_found_ceps = set()

# Dicionário para mapear CEPs antigos para o novo CEP retornado pela API (quando houver alteração)
changed_ceps = {}

def get_address(cep):
    """
    Consulta a API ViaCEP para obter CEP atualizado, bairro e cidade a partir do CEP informado.
    Retorna uma tupla (cep_atualizado, bairro, cidade).
    Tenta reconsultar a API até 3 vezes caso não obtenha sucesso.
    """
    # Converte o CEP para string, remove hífens e espaços
    original_cep = str(cep).replace("-", "").strip()
    # Garante que o CEP possui 8 dígitos, preenchendo com zeros à esquerda se necessário
    original_cep = original_cep.zfill(8)
    
    # Se já consultou esse CEP, retorna do cache
    if original_cep in cache:
        return cache[original_cep]
    
    max_attempts = 3
    result = (None, None, None)
    for attempt in range(max_attempts):
        try:
            url = f"https://viacep.com.br/ws/{original_cep}/json/"
            response = requests.get(url)
            if response.status_code == 200:
                data = response.json()
                if "erro" not in data:
                    # Extrai o CEP atualizado da resposta, se houver, e padroniza
                    new_cep = data.get("cep", original_cep)
                    new_cep = new_cep.replace("-", "").strip().zfill(8)
                    
                    # Verifica se o CEP retornado é diferente do original
                    if new_cep != original_cep:
                        changed_ceps[original_cep] = new_cep
                    
                    result = (new_cep, data.get("bairro"), data.get("localidade"))
                    break  # Sai do loop se a consulta for bem-sucedida
                else:
                    result = (None, None, None)
            else:
                result = (None, None, None)
        except Exception as e:
            print(f"Erro ao consultar o CEP {original_cep}, tentativa {attempt+1}: {e}")
            result = (None, None, None)
        # Aguarda 1 segundo antes de tentar novamente
        time.sleep(1)
    
    # Armazena o resultado no cache
    cache[original_cep] = result
    
    # Se após as tentativas o CEP não foi encontrado, adiciona ao conjunto de não encontrados
    if result == (None, None, None):
        not_found_ceps.add(original_cep)
        
    return result

# Lê o arquivo Excel, garantindo que a coluna do CEP seja interpretada como string
df = pd.read_excel('Clientes Gerais.xlsx', dtype={'Código postal (CEP) de envio': str})

# Garante que os CEPs estão sem espaços extras
df["Código postal (CEP) de envio"] = df["Código postal (CEP) de envio"].apply(lambda x: str(x).strip())

# Cria as novas colunas "CEP Atualizado", "Bairro" e "Cidade" aplicando a função get_address em cada CEP
df[["CEP Atualizado", "Bairro", "Cidade"]] = df["Código postal (CEP) de envio"].apply(lambda cep: pd.Series(get_address(cep)))

# Exporta o DataFrame final para um arquivo Excel (.xlsx)
df.to_excel("Relatorio_de_vendas_com_endereco.xlsx", index=False)

# Printa um aviso se houver CEPs que não foram encontrados após as tentativas
if not_found_ceps:
    print("CEP(s) não encontrados após tentativas:", ", ".join(sorted(not_found_ceps)))
else:
    print("Todos os CEPs foram encontrados.")

# Informa quais CEPs foram alterados (ou seja, cujo CEP atualizado é diferente do CEP original)
if changed_ceps:
    print("Os seguintes CEPs foram alterados pelos Correios:")
    for old, new in sorted(changed_ceps.items()):
        print(f"CEP antigo: {old} -> CEP novo: {new}")
else:
    print("Nenhum CEP foi alterado pelos Correios.")

print("Processamento finalizado. Planilha atualizada salva como 'Relatorio_de_vendas_com_endereco.xlsx'.")


Erro ao consultar o CEP 05630050, tentativa 1: ('Connection aborted.', ConnectionResetError(10054, 'Foi forçado o cancelamento de uma conexão existente pelo host remoto', None, 10054, None))
Erro ao consultar o CEP 05630050, tentativa 2: ('Connection aborted.', ConnectionAbortedError(10053, 'Uma conexão estabelecida foi anulada pelo software no computador host', None, 10053, None))
Erro ao consultar o CEP 05630050, tentativa 3: HTTPSConnectionPool(host='viacep.com.br', port=443): Max retries exceeded with url: /ws/05630050/json/ (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x00000243D26CAAD0>: Failed to resolve 'viacep.com.br' ([Errno 11001] getaddrinfo failed)"))
Erro ao consultar o CEP 00030000, tentativa 1: HTTPSConnectionPool(host='viacep.com.br', port=443): Max retries exceeded with url: /ws/00030000/json/ (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x00000243D26CA5D0>: Failed to resolve 'viacep.com.br' ([Errno 110

In [3]:
import pandas as pd
import re

def clean_money(value):
    """
    Limpa e converte valores monetários que possam vir com "R$" e separadores de milhar.
    Exemplo: "R$ 1.234,56" é convertido para 1234.56 (float).
    """
    if isinstance(value, str):
        # Remove o símbolo de moeda e espaços
        value = value.replace("R$", "").strip()
        # Remove pontos (separador de milhar) e converte a vírgula decimal para ponto
        value = value.replace(".", "").replace(",", ".")
    try:
        return float(value)
    except Exception as e:
        print(f"Erro na conversão do valor '{value}': {e}")
        return 0.0

# Carrega a planilha com os dados.
df = pd.read_excel("Relatorio_de_vendas_com_endereco.xlsx")

# Verifica se as colunas necessárias existem no DataFrame.
colunas_necessarias = ['Bairro', 'Cidade', 'Total de pedidos', 'Total de vendas']
if not all(col in df.columns for col in colunas_necessarias):
    raise ValueError("O arquivo não contém todas as colunas necessárias: 'Bairro', 'Cidade', 'Total de pedidos' e 'Total de vendas'.")

# Cria a nova coluna "Bairro-Cidade" concatenando as colunas Bairro e Cidade.
df["Bairro-Cidade"] = df["Bairro"].astype(str) + "-" + df["Cidade"].astype(str)

# Limpa e converte a coluna 'Total de vendas'
if df['Total de vendas'].dtype == object:
    df['Total de vendas'] = df['Total de vendas'].apply(clean_money)

# Se a coluna 'Total de pedidos' estiver como string, converte para numérico removendo caracteres não numéricos.
if df['Total de pedidos'].dtype == object:
    df['Total de pedidos'] = pd.to_numeric(df['Total de pedidos'].str.replace(r'\D+', '', regex=True), errors='coerce')

# Agrupa por "Bairro-Cidade" para obter a soma dos 'Total de pedidos'
df_quantidade = df.groupby("Bairro-Cidade", as_index=False)['Total de pedidos'].sum()
# Ordena decrescentemente: os bairros com maior quantidade de pedidos aparecem primeiro.
df_quantidade = df_quantidade.sort_values(by='Total de pedidos', ascending=False)

# Agrupa por "Bairro-Cidade" para obter a soma dos 'Total de vendas'
df_valor = df.groupby("Bairro-Cidade", as_index=False)['Total de vendas'].sum()
# Ordena decrescentemente: os bairros com maior valor total de vendas aparecem primeiro.
df_valor = df_valor.sort_values(by='Total de vendas', ascending=False)

# Exporta as duas tabelas agrupadas para um único arquivo Excel com duas planilhas:
#  - "Quantidade": Tabela com a soma de 'Total de pedidos'
#  - "Valor": Tabela com a soma de 'Total de vendas'
with pd.ExcelWriter("Relatorio_agrupado.xlsx") as writer:
    df_quantidade.to_excel(writer, sheet_name="Quantidade", index=False)
    df_valor.to_excel(writer, sheet_name="Valor", index=False)

print("Processamento finalizado. Arquivo 'Relatorio_agrupado.xlsx' criado com as duas planilhas.")


Processamento finalizado. Arquivo 'Relatorio_agrupado.xlsx' criado com as duas planilhas.


In [4]:
import pandas as pd

# Lê o arquivo Excel já processado com as colunas "Bairro" e "Cidade"
df = pd.read_excel("Relatorio_de_vendas_com_endereco.xlsx")

# Filtra somente os registros cuja cidade seja "São Paulo"
# Utilizamos .str.strip() para remover espaços extras e comparamos com "São Paulo"
df_sp = df[df["Cidade"].str.strip() == "São Paulo"]

# Salva o DataFrame filtrado em um novo arquivo Excel
df_sp.to_excel("Relatorio_de_vendas_SaoPaulo.xlsx", index=False)

print("Planilha com registros de São Paulo salva como 'Relatorio_de_vendas_SaoPaulo.xlsx'.")


Planilha com registros de São Paulo salva como 'Relatorio_de_vendas_SaoPaulo.xlsx'.


In [ ]:
import pandas as pd

# Lê o arquivo Excel já processado com as colunas "Bairro" e "Cidade"
df = pd.read_excel("Relatorio_de_vendas_com_endereco.xlsx")

# Filtra somente os registros cuja cidade seja "São Paulo"
# Utilizamos .str.strip() para remover espaços extras e comparamos com "São Paulo"
df_sp = df[df["Cidade"].str.strip() == "São Paulo"]

# Salva o DataFrame filtrado em um novo arquivo Excel
df_sp.to_excel("Relatorio_de_vendas_SaoPaulo.xlsx", index=False)

print("Planilha com registros de São Paulo salva como 'Relatorio_de_vendas_SaoPaulo.xlsx'.")


Planilha com registros de São Paulo salva como 'Relatorio_de_vendas_SaoPaulo.xlsx'.
